In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [11]:
pd.options.display.max_columns = 99
pd.options.display.max_rows = 1000
#pd.set_option('display.max_rows', 1000)
train = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv.gz')
train

In [44]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8,6))
sns.barplot(train['MSZoning'],train['SalePrice'])

In [13]:
test = pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv.gz')
test

In [14]:
all_data = pd.concat([train,test])
all_data

In [15]:
all_data_2 = all_data.drop(columns=['SalePrice','PoolQC','MiscFeature','Id'])
all_data_2

In [18]:
all_data_2.columns

In [19]:
numericla_columns = [i for i in all_data_2.columns if all_data_2[i].dtype in ['int64', 'float64']]
len(numericla_columns)

In [20]:
categorical_columns = [i for i in all_data_2.columns if all_data_2[i].dtype=='object']
len(categorical_columns)

In [21]:
cat = all_data_2.columns[all_data_2.dtypes==object]
cat

In [22]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for i in cat:
    all_data_2[i] = le.fit_transform(list(all_data_2[i]))

In [23]:
all_data_2

In [24]:
train_2 = all_data_2[:len(train)]
test_2 = all_data_2[len(train):]

In [25]:
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

#xgb = XGBRegressor(random_state=0)
cbr = CatBoostRegressor(verbose=200, task_type='GPU')
cbr.fit(train_2, train['SalePrice'])
#xgb.fit(train_2, train['SalePrice'])

In [26]:
result = cbr.predict(test_2)
result

In [27]:
pd.Series(cbr.feature_importances_, index=train_2.columns).sort_values(ascending=False)

In [28]:
sub = pd.read_csv('/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz')
sub

In [29]:
sub['SalePrice'] = result
sub

In [30]:
sub.to_csv('sub.csv', index=False)